# Module 02: Model Versioning and Registry

**Difficulty**: ⭐⭐ Intermediate
**Estimated Time**: 60 minutes
**Prerequisites**: 
- [Module 01: Experiment Tracking with MLflow](01_experiment_tracking_mlflow.ipynb)
- Understanding of ML model lifecycle

## Learning Objectives
By the end of this notebook, you will be able to:
1. Understand the need for model versioning and registry
2. Set up and configure MLflow Model Registry
3. Register models to the MLflow Model Registry
4. Transition models between stages (None → Staging → Production → Archived)
5. Track model lineage and metadata
6. Load and use registered models for predictions

## 1. Why Model Versioning and Registry?

### The Problem Without a Model Registry

Imagine your ML team has trained dozens of models:

- **Data Scientist**: "I deployed model_v3_final_FINAL_2.pkl to production last week"
- **MLOps Engineer**: "Which Python version? Which scikit-learn version? Which training data?"
- **Product Manager**: "We need to rollback! The model is making bad predictions!"
- **Data Scientist**: "Uh... let me search my email for the S3 link... 😰"

### The Solution: MLflow Model Registry

The **MLflow Model Registry** is a centralized model store that provides:

1. **Version Control**: Track all versions of a model over time
2. **Stage Management**: Move models through dev → staging → production
3. **Lineage Tracking**: Know which data and code produced each model
4. **Annotations**: Add descriptions, tags, and metadata
5. **Model Discovery**: Find the right model quickly
6. **Governance**: Control who can promote models to production

### Model Stages

MLflow defines these standard stages:

- **None**: Initial state when model is first registered
- **Staging**: Model is being tested in a staging environment
- **Production**: Model is live and serving predictions
- **Archived**: Model is no longer in use but kept for historical reference

In [ ]:
# Setup: Import all required libraries
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning libraries
from sklearn.datasets import make_classification, load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# MLflow
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
from mlflow.entities.model_registry import ModelVersion

# Set random seed for reproducibility
np.random.seed(42)

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

print(f"MLflow version: {mlflow.__version__}")
print("Setup complete!")

## 2. Setting Up MLflow Model Registry

The Model Registry requires a backend database to store model metadata. For this tutorial, we'll use a local SQLite database.

In [ ]:
# Set up MLflow with a backend store that supports Model Registry
# SQLite is used for simplicity; production systems use PostgreSQL or MySQL
mlflow.set_tracking_uri("sqlite:///mlflow.db")

# Create MLflow client for interacting with the registry
client = MlflowClient()

print(f"Tracking URI: {mlflow.get_tracking_uri()}")
print("Model Registry is now available!")

## 3. Preparing Data and Training Models

Let's create a dataset and train multiple models to demonstrate versioning.

In [ ]:
# Generate synthetic classification dataset
X, y = make_classification(
    n_samples=1000,
    n_features=20,
    n_informative=15,
    n_redundant=5,
    n_classes=2,
    random_state=42
)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"Class distribution: {np.bincount(y_train)}")

## 4. Registering Your First Model

There are two ways to register a model:
1. **During training**: Use `mlflow.sklearn.log_model()` with `registered_model_name`
2. **After training**: Use `mlflow.register_model()` with a run ID

We'll demonstrate both approaches.

In [ ]:
# Approach 1: Register model during training

mlflow.set_experiment("model_registry_demo")

with mlflow.start_run(run_name="random_forest_v1") as run:
    
    # Train model
    params = {"n_estimators": 100, "max_depth": 10, "random_state": 42}
    mlflow.log_params(params)
    
    model = RandomForestClassifier(**params)
    model.fit(X_train, y_train)
    
    # Evaluate
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)
    
    # Log and register model in one step
    # This creates a new registered model named "binary_classifier"
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="model",
        registered_model_name="binary_classifier"  # This registers the model!
    )
    
    print(f"✓ Model registered as 'binary_classifier' version 1")
    print(f"  Run ID: {run.info.run_id}")
    print(f"  Accuracy: {accuracy:.4f}")

## 5. Viewing Registered Models

Let's explore what was created in the registry.

In [ ]:
# List all registered models
registered_models = client.search_registered_models()

print(f"Total registered models: {len(registered_models)}\n")

for rm in registered_models:
    print(f"Model: {rm.name}")
    print(f"  Description: {rm.description or 'No description'}")
    print(f"  Latest versions: {len(rm.latest_versions)}")
    print()

In [ ]:
# Get detailed information about a specific model
model_name = "binary_classifier"
model_versions = client.search_model_versions(f"name='{model_name}'")

print(f"Model: {model_name}")
print(f"Total versions: {len(model_versions)}\n")

for mv in model_versions:
    print(f"Version {mv.version}:")
    print(f"  Stage: {mv.current_stage}")
    print(f"  Run ID: {mv.run_id}")
    print(f"  Created: {pd.to_datetime(mv.creation_timestamp, unit='ms')}")
    print()

## 6. Adding Model Descriptions and Tags

Make your models discoverable by adding rich metadata.

In [ ]:
# Update the registered model description
client.update_registered_model(
    name="binary_classifier",
    description="Binary classification model for customer churn prediction. "
                "Uses Random Forest with optimized hyperparameters."
)

# Add tags to the model version
client.set_model_version_tag(
    name="binary_classifier",
    version="1",
    key="task",
    value="classification"
)

client.set_model_version_tag(
    name="binary_classifier",
    version="1",
    key="framework",
    value="scikit-learn"
)

client.set_model_version_tag(
    name="binary_classifier",
    version="1",
    key="training_data",
    value="synthetic_v1"
)

print("✓ Model description and tags updated!")

## 7. Model Stages: Moving Through the Lifecycle

Now let's transition our model through different stages.

In [ ]:
# Transition model to Staging
# This indicates the model is ready for testing in a staging environment

client.transition_model_version_stage(
    name="binary_classifier",
    version=1,
    stage="Staging",
    archive_existing_versions=False  # Keep other staged versions
)

print("✓ Model transitioned to Staging")

# Verify the stage change
model_version = client.get_model_version(name="binary_classifier", version="1")
print(f"  Current stage: {model_version.current_stage}")

## 8. Registering a New Version

Let's train an improved model and register it as version 2.

In [ ]:
# Train an improved model with better hyperparameters

with mlflow.start_run(run_name="random_forest_v2_improved") as run:
    
    # Better hyperparameters based on tuning
    params = {
        "n_estimators": 200,
        "max_depth": 15,
        "min_samples_split": 5,
        "random_state": 42
    }
    mlflow.log_params(params)
    
    model = RandomForestClassifier(**params)
    model.fit(X_train, y_train)
    
    # Evaluate
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)
    
    # Register as version 2
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="model",
        registered_model_name="binary_classifier"  # Same name, new version!
    )
    
    # Add version-specific description
    client.update_model_version(
        name="binary_classifier",
        version="2",
        description="Improved version with deeper trees and more estimators"
    )
    
    print(f"✓ Model registered as 'binary_classifier' version 2")
    print(f"  Accuracy: {accuracy:.4f}")

## 9. Comparing Model Versions

Let's compare the performance of different versions.

In [ ]:
# Get all versions and their metrics
model_versions = client.search_model_versions(f"name='binary_classifier'")

version_data = []
for mv in model_versions:
    # Get the run to access metrics
    run = client.get_run(mv.run_id)
    
    version_data.append({
        "version": mv.version,
        "stage": mv.current_stage,
        "accuracy": run.data.metrics.get("accuracy", 0),
        "created": pd.to_datetime(mv.creation_timestamp, unit='ms'),
        "run_id": mv.run_id[:8]  # Shortened for display
    })

# Create comparison DataFrame
comparison_df = pd.DataFrame(version_data).sort_values("version")
print("Model Version Comparison:")
print(comparison_df.to_string(index=False))

# Visualize accuracy across versions
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(comparison_df['version'], comparison_df['accuracy'], 
        marker='o', linewidth=2, markersize=10, color='darkblue')
ax.set_xlabel('Model Version', fontweight='bold', fontsize=12)
ax.set_ylabel('Accuracy', fontweight='bold', fontsize=12)
ax.set_title('Model Accuracy Across Versions', fontweight='bold', fontsize=14)
ax.grid(alpha=0.3)
ax.set_ylim(0.85, 1.0)
plt.tight_layout()
plt.show()

## 10. Promoting Model to Production

After validating version 2 in staging, let's promote it to production.

In [ ]:
# Move version 2 to Production
# archive_existing_versions=True will archive the current production model

client.transition_model_version_stage(
    name="binary_classifier",
    version=2,
    stage="Production",
    archive_existing_versions=True  # Archive old production models
)

print("✓ Version 2 promoted to Production")

# Check what's now in production
prod_models = client.get_latest_versions("binary_classifier", stages=["Production"])
for model in prod_models:
    print(f"  Production model: Version {model.version}")

## 11. Loading Models from Registry

The registry makes it easy to load models by stage or version.

In [ ]:
# Method 1: Load by stage (recommended for applications)
# This always gets the latest production model
model_uri = f"models:/binary_classifier/Production"
production_model = mlflow.sklearn.load_model(model_uri)

print("✓ Loaded production model")
print(f"  URI: {model_uri}")

# Test the loaded model
sample_predictions = production_model.predict(X_test[:5])
print(f"\nSample predictions: {sample_predictions}")
print(f"Actual values: {y_test[:5]}")

In [ ]:
# Method 2: Load by specific version
# Useful for comparing versions or debugging
model_uri_v1 = "models:/binary_classifier/1"
model_v1 = mlflow.sklearn.load_model(model_uri_v1)

model_uri_v2 = "models:/binary_classifier/2"
model_v2 = mlflow.sklearn.load_model(model_uri_v2)

# Compare predictions
pred_v1 = model_v1.predict(X_test)
pred_v2 = model_v2.predict(X_test)

acc_v1 = accuracy_score(y_test, pred_v1)
acc_v2 = accuracy_score(y_test, pred_v2)

print("Version Comparison:")
print(f"  Version 1 accuracy: {acc_v1:.4f}")
print(f"  Version 2 accuracy: {acc_v2:.4f}")
print(f"  Improvement: {(acc_v2 - acc_v1)*100:.2f}%")

## 12. Model Lineage and Metadata

The registry maintains full lineage: which run created the model, when, and with what parameters.

In [ ]:
# Get full lineage for production model
prod_model = client.get_latest_versions("binary_classifier", stages=["Production"])[0]
run = client.get_run(prod_model.run_id)

print(f"Production Model Lineage:")
print(f"  Version: {prod_model.version}")
print(f"  Run ID: {prod_model.run_id}")
print(f"  Created: {pd.to_datetime(prod_model.creation_timestamp, unit='ms')}")
print(f"\nTraining Parameters:")
for key, value in run.data.params.items():
    print(f"  {key}: {value}")

print(f"\nMetrics:")
for key, value in run.data.metrics.items():
    print(f"  {key}: {value:.4f}")

print(f"\nTags:")
for key, value in prod_model.tags.items():
    print(f"  {key}: {value}")

## 13. Archiving Old Models

Keep your registry clean by archiving models that are no longer needed.

In [ ]:
# Archive version 1 (it's in Staging and we have a better production model)
client.transition_model_version_stage(
    name="binary_classifier",
    version=1,
    stage="Archived"
)

print("✓ Version 1 archived")

# View all versions and their stages
all_versions = client.search_model_versions("name='binary_classifier'")
print("\nAll Model Versions:")
for mv in sorted(all_versions, key=lambda x: x.version):
    print(f"  Version {mv.version}: {mv.current_stage}")

## 14. Exercises

Test your understanding of model versioning and registry.

### Exercise 1: Register a Logistic Regression Model

Train a LogisticRegression model and register it under the name "logistic_classifier".

**Requirements**:
1. Train a LogisticRegression model on the same dataset
2. Log parameters and accuracy
3. Register the model as "logistic_classifier"
4. Add a description and at least 2 tags

In [ ]:
# Exercise 1: Your code here

# YOUR CODE HERE

In [ ]:
# Exercise 1 Solution

with mlflow.start_run(run_name="logistic_regression_v1"):
    
    # Train model
    params = {"max_iter": 1000, "C": 1.0, "random_state": 42}
    mlflow.log_params(params)
    
    model = LogisticRegression(**params)
    model.fit(X_train, y_train)
    
    # Evaluate
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)
    
    # Register model
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="model",
        registered_model_name="logistic_classifier"
    )
    
    print(f"✓ Logistic classifier registered (Accuracy: {accuracy:.4f})")

# Add description and tags
client.update_registered_model(
    name="logistic_classifier",
    description="Baseline logistic regression model for binary classification"
)

client.set_model_version_tag(
    name="logistic_classifier", version="1", key="model_type", value="logistic"
)

client.set_model_version_tag(
    name="logistic_classifier", version="1", key="baseline", value="true"
)

print("✓ Description and tags added")

### Exercise 2: Transition Through Stages

Practice moving a model through the complete lifecycle.

**Requirements**:
1. Transition "logistic_classifier" version 1 to Staging
2. Check its current stage
3. Transition it to Production
4. Verify the transition

In [ ]:
# Exercise 2 Solution

# Move to Staging
client.transition_model_version_stage(
    name="logistic_classifier",
    version=1,
    stage="Staging"
)
print("✓ Transitioned to Staging")

# Check current stage
model_version = client.get_model_version(name="logistic_classifier", version="1")
print(f"  Current stage: {model_version.current_stage}")

# Move to Production
client.transition_model_version_stage(
    name="logistic_classifier",
    version=1,
    stage="Production"
)
print("✓ Transitioned to Production")

# Verify
model_version = client.get_model_version(name="logistic_classifier", version="1")
print(f"  Final stage: {model_version.current_stage}")

### Exercise 3: Load and Compare Models

Load both production models and compare their performance.

**Requirements**:
1. Load the production version of "binary_classifier"
2. Load the production version of "logistic_classifier"
3. Make predictions on the test set
4. Print which model has better accuracy

In [ ]:
# Exercise 3 Solution

# Load both production models
rf_model = mlflow.sklearn.load_model("models:/binary_classifier/Production")
lr_model = mlflow.sklearn.load_model("models:/logistic_classifier/Production")

# Make predictions
rf_pred = rf_model.predict(X_test)
lr_pred = lr_model.predict(X_test)

# Calculate accuracies
rf_accuracy = accuracy_score(y_test, rf_pred)
lr_accuracy = accuracy_score(y_test, lr_pred)

# Compare
print("Production Model Comparison:")
print(f"  Random Forest accuracy: {rf_accuracy:.4f}")
print(f"  Logistic Regression accuracy: {lr_accuracy:.4f}")
print()

if rf_accuracy > lr_accuracy:
    print(f"✓ Random Forest is better by {(rf_accuracy - lr_accuracy)*100:.2f}%")
else:
    print(f"✓ Logistic Regression is better by {(lr_accuracy - rf_accuracy)*100:.2f}%")

# Visualize comparison
fig, ax = plt.subplots(figsize=(8, 6))
models = ['Random Forest', 'Logistic Regression']
accuracies = [rf_accuracy, lr_accuracy]
colors = ['skyblue', 'lightcoral']

ax.bar(models, accuracies, color=colors, edgecolor='black', alpha=0.7)
ax.set_ylabel('Accuracy', fontweight='bold', fontsize=12)
ax.set_title('Production Models Comparison', fontweight='bold', fontsize=14)
ax.set_ylim(0.85, 1.0)
ax.grid(axis='y', alpha=0.3)

for i, v in enumerate(accuracies):
    ax.text(i, v + 0.005, f'{v:.4f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

## 15. Summary

### Key Takeaways

1. **MLflow Model Registry** provides centralized model versioning and lifecycle management

2. **Model stages** (None → Staging → Production → Archived) represent the deployment lifecycle

3. **Registering models** can be done during training or after using run IDs

4. **Rich metadata** (descriptions, tags, lineage) makes models discoverable and traceable

5. **Loading models** by stage ensures applications always use the right version

6. **Version comparison** helps teams make data-driven promotion decisions

### Best Practices

- Always add descriptions to registered models
- Use tags to organize models by team, project, or data source
- Test models in Staging before promoting to Production
- Archive old models rather than deleting them
- Load models by stage in production applications
- Document the criteria for promoting models

### What's Next?

In **Module 03**, we'll explore:
- **Model serialization** formats (pickle, joblib, ONNX)
- **Cross-platform compatibility** considerations
- **Model size optimization**
- **Best practices** for saving and loading models

## 16. Additional Resources

### Documentation
- **MLflow Model Registry**: https://mlflow.org/docs/latest/model-registry.html
- **Model Registry API**: https://mlflow.org/docs/latest/python_api/mlflow.client.html
- **Transitioning Models**: https://mlflow.org/docs/latest/model-registry.html#transitioning-an-mlflow-models-stage

### Tutorials
- **MLflow Registry Tutorial**: https://mlflow.org/docs/latest/registry.html
- **Model Registry Workflows**: https://mlflow.org/docs/latest/model-registry.html#model-registry-workflows

### Advanced Topics
- Setting up model registry with PostgreSQL/MySQL
- Implementing approval workflows for production
- Integrating registry with CI/CD pipelines
- Model governance and access control